In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import time
import random
import copy
import math


In [7]:
# 现在要对5个州做一个总的用水用电的水分配方案（分别给多少水，给多少电 -> 存多少水）
# a = 5个州的分别的总用水需求，b = 5个州分别的用电需求 -> 转化为5个州的存水需求 w
# 求最优的a、w，再计算得到最优的a、b
# 最优的评判：满足总和小于等于 M_v + P_v 的前提下，得分score()最高
# A = 5州分别实际总用水， B = 5州分别实际总用电
# (2019 data)
M_v = 4207.6251 - 0.402866840525917
P_v = 3212.9429 - 192.830940038404
N = int((M_v*1e4 + P_v*1e4)/3.22321112134556)
bitlength = N.bit_length()
A = eval("""[4632.39
20171.5
7849.86
2203.23
9412.49]""".replace('\n', ','))
B = eval("""316.5298945
480.7151231
147.2743259
94.57796014
151.905041""".replace('\n', ','))
A, B = np.array(A), np.array(B)
A = A / 1e3 * 1e4
B = B / 365

N_GENERATIONS=10000

In [8]:
# 因为目前只有两个水坝分别的水电转换的关系式，应该需要再拟合一个总的： y = 1.1370 lnx + -4.4675
def w2e(x): return 1.1370 * np.log(x) + -4.4675
def e2w(y): return np.exp((y + 4.4675) / 1.1370)
print("the corresponding water storage to meet elec requirement is", e2w(np.sum(B)))
ratio = B/sum(B)
a_prime = np.append(A, e2w(np.sum(B)) * 1e4 * ratio)
print(a_prime/1e4)
print(N / 1e4)
print(sum(a_prime)/1e4)
m=1
def score(ai, a_primei, m):
    value = m / a_primei - m / ai
    return value

def evaluate(a, a_prime, m):
    product = 0
    for i in range(len(a)):
        product = product + score(a[i], a_prime[i], m)
    return product / len(a)


the corresponding water storage to meet elec requirement is 896.9949565709472
[  4.63239     20.1715       7.84986      2.20323      9.41249
 238.39224184 362.0471806  110.91861251  71.23071892 114.4062027 ]
2242.2776
941.2644265709471


In [9]:
def init_random():
    x, tmp = [], []
    while len(tmp) < 9:
        r = random.randint(0, N)
        if r not in tmp and r != 0 and r != N:
            tmp.append(r)
    tmp.sort()
    x.append(tmp[0])
    for i in range(1, 9):
        x.append(tmp[i]-tmp[i-1])
    x.append(N-tmp[-1])
    return x

def swap(x):
    new_x = copy.copy(x)
    x_copy = copy.copy(x)
    x = list(x)
    np.random.shuffle(x_copy)
    bitlength = N.bit_length()
    s = random.randint(0, bitlength-1)
    new_x[x.index(x_copy[0])] = int((bin(x_copy[0])[2:]).zfill(bitlength)[:s] + bin(x_copy[1])[2:].zfill(bitlength)[s:],2)
    new_x[x.index(x_copy[1])] = int(bin(x_copy[1])[2:].zfill(bitlength)[:s] + bin(x_copy[0])[2:].zfill(bitlength)[s:],2)
    return new_x

def mutation(x):
    new_x = copy.copy(x)
    x_copy = copy.copy(x)
    ridx = random.randint(0, len(x_copy)-1)
    r = random.randint(0, bitlength-1)
    x_copy_bin = (bin(x_copy[ridx])[2:]).zfill(bitlength)
    x_copy_bin = x_copy_bin[:r] + str((int(x_copy_bin[r])+1) % 2) + x_copy_bin[r+1:]
    new_x[ridx] = int(x_copy_bin, 2)
    return new_x

def legal(x):
    if 0 in x:
        return False
    s = sum(x)
    if s > N:
        return False
    else:
        return True

In [10]:
best_score = -999999
best = []
next_x = init_random()
for _ in range(N_GENERATIONS):
    x = next_x
    x_score = evaluate(x, a_prime, m)
    if x_score > best_score:
        best_score = x_score
        best = x
    y = swap(x)
    if not legal(y):
        y = x
    y_score = evaluate(y, a_prime, m)
    if y_score > best_score:
        best_score = y_score
        best = y
    z = mutation(y)
    if not legal(z):
        z = y
    z_score = evaluate(z, a_prime, m)
    if z_score > best_score:
        best_score = z_score
        best = z
    next_x = best
best = np.array(best)
print("result is ")
print("-------------------------")
print("The demand is ", a_prime/1e4)
print("The real distribution is ", best/1e4)
print("The best score is " ,best_score * 1e4)
print([score(best[i], a_prime[i], m) * 1e4 for i in range(len(best))])
print(abs(np.sum(best) - N)/1e4)

result is 
-------------------------
The demand is  [  4.63239     20.1715       7.84986      2.20323      9.41249
 238.39224184 362.0471806  110.91861251  71.23071892 114.4062027 ]
The real distribution is  [207.7182 207.7183 226.0992 222.8228 181.8623 213.0166 157.2863 222.8228
 183.5007 419.4304]
The best score is  0.09442982796191357
[0.21105707425998194, 0.04476068294797619, 0.12296796956038555, 0.4493912064565426, 0.10074314699154965, -0.000499702439394714, -0.003595761868441889, 0.004527748367493018, 0.008589316312806034, 0.006356599030236834]
0.0
